In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from dummyPy import OneHotEncoder
import random
import datetime
import pickle  # to store temporary variable
from imblearn.ensemble import EasyEnsemble
%matplotlib inline

In [2]:
train_ori = './train.csv'
test_ori = './test.csv'
train_fea = './train_sam_after_features.csv'
test_fea = './test_after_features.csv'
train_sample = './train_sample.csv'

In [3]:
df = pd.read_csv(train_ori,chunksize=100000)

In [6]:
for chunk in df:
    sample = chunk.sample(n=None,frac=0.10,axis=0)
    print(sample['id'].apply(lambda x: '{:.0f}'.format(x)))
    break

283183    12159484766517956608
255713     9914249898422147072
277389    11688053272782520320
217218     5285064114893029376
221623     5815022459870647296
228991     6693192595504628736
236390     7575878423142557696
204335     3712427981726143488
274944    11491174947922042880
258087    10128026526901188608
205500     3855691699402215424
278735    11793727306055471104
257777    10103031060909678592
229978     6812537602970500096
218433     5430826808706338816
286937    12461687815028246528
227777      654769310593129984
238305     7800929155052729344
276990    11656129286479917056
232025     7063000843234208768
295081    13108386936957042688
223710     6059571629623580672
296789    13245397918574764032
247378     8903397172094860288
272668    11310442183958323200
299225    13443433781244772352
284357     1225507994828405760
205897      390607502523173376
213225     4801592258214827008
274220    11434268058434613248
                  ...         
292927    12943914354563540992
294983  

In [48]:
count  = 0
index = 0
with open(train_sample,"a") as samplefile:
    for chunk in df:
        sample = chunk.sample(n=None,frac=0.10,axis=0)
        sample["id"] = sample["id"].apply(lambda x: '{:.0f}'.format(x))
        if index==0:
            sample.to_csv(samplefile, index=False, header=True)
        else:
            sample.to_csv(samplefile, index=False, header=False)
        index +=1
        print(index)
        #print(chunk.columns)

        count = count + chunk.shape[0]
    print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [3]:
def getweekday(x):
    '''
    :param x: YYMMDD
    :return: weekday
    '''
    sdt = str(x)
    year = int(sdt[0:2])
    month = int(sdt[2:4])
    day = int(sdt[4:6])
    dt = datetime.date(year, month, day)
    weekday = dt.weekday()
    return weekday

def time_period(hour):
    period = 0
    hour = int(hour)
    if hour>0 and hour <6:
        period = 1
    elif hour>=6 and hour<11 :
        period = 2
    elif hour>=11 and hour<13:
        period = 3
    elif hour>13 and hour<17:
        period = 4
    elif hour>=17 and hour<24:
        period = 5
    return period

def create_feature(data):
    data["size"] = data["C15"] * data["C16"]
    # 将hour列拆分为
    data["hour1"] = np.round(data.hour % 100)
    data["weekday"] = data["hour"].map(lambda x: getweekday(x))
    data["time_period"] = data["hour1"].map(lambda x:time_period(x))
    data["app_site_id"] = data["app_id"] + "_" + data["site_id"]
    data["app_site_id_model"] = data["app_site_id"] + "_" + data["device_model"]
    return data

In [4]:
df_train_org = pd.read_csv(train_sample, index_col=None, chunksize=500000, iterator=True)
df_test_org  = pd.read_csv(test_ori, index_col=None, chunksize=500000, iterator=True)

In [ ]:
%%time
# train set
hd_flag = True  # add column names at 1-st row
for chunk in df_train_org:
    df = chunk.copy()
    df = create_feature(df)
    with open(train_fea, 'a') as f:
        df.to_csv(f,header = hd_flag, index = False)
    hd_flag = False

In [ ]:
%%time


## test set
hd_flag = True  # add column names at 1-st row
for chunk in df_test_org:
    df = chunk.copy()
    df = create_feature(df)
    with open(test_fea, 'a') as f:
        df.to_csv(f,header = hd_flag, index = False)      
    hd_flag = False 

In [3]:
%%time
train_lb = pd.read_csv(train_ori)

Wall time: 6min 48s


In [4]:
label_encode = './label_encode_all'
cols = ['C1', 
        'banner_pos', 
        'device_type', 
        'device_conn_type',
        'C14', 
        'C17',
       'C18','C19','C20','C21']

In [5]:
%%time
le = {}
for col in cols:
    le[col] = LabelEncoder()
    le[col].fit(train_lb[col])

pickle.dump(le,open(label_encode,'wb'))

Wall time: 22.4 s
